In [ ]:
import torch
from utils.utils import HadamardTransform
from utils.hadamard_utils import matmul_hadU_cuda, get_hadK
import math
import numpy as np
import struct

In [ ]:
# Function to get the bit pattern of a floating-point number
def float_to_hex(f):
    # Pack the float into 4 bytes (32-bit float) or 8 bytes (64-bit float)
    packed = struct.pack('>f', f)  # Use '>f' for 32-bit float, '>d' for 64-bit double
    # Unpack as an integer to get the bit pattern
    hex_value = ''.join(f'{byte:02x}' for byte in packed)
    return hex_value

In [ ]:
q = torch.rand(2,2,11008, dtype=torch.float32, device='cuda')
q_pad = torch.concat([q, torch.zeros(q.shape[0], q.shape[1], 5376, dtype=q.dtype, device="cuda")], dim=-1)

# q = torch.rand(2,2,11008, dtype=torch.float32)
# q_pad = torch.concat([q, torch.zeros(q.shape[0], q.shape[1], 5376, dtype=q.dtype)], dim=-1)

np.log2([11008])
print(2**14)
print(16384-11008)

dtype = q.dtype
q_ref = (HadamardTransform.apply(q.float()) / math.sqrt(q.shape[-1])).to(dtype)
q_evl = (HadamardTransform.apply(q_pad.float()) / math.sqrt(q.shape[-1])).to(dtype)

print((q_ref-q_evl[...,:11008]).abs().mean())
print(((q_ref-q_evl[...,:11008]).abs()/(q_ref.abs() + 1e-6)).abs().mean())

In [ ]:
n1, n2, n3 = 2, 2, 4*172
q = torch.zeros(n1*n2*n3, dtype=torch.float32)
for i in range(n1*n2*n3):
  q[i] = i/(n1*n2*n3)
q = q.reshape(n1, n2, n3)


had_K, K = get_hadK(q.shape[-1])
q_had = matmul_hadU_cuda(q, had_K, K)

print(had_K)
print(K)
print(q_had)

data = q_had.numpy()
data.tofile('./hadamard_test/hadamard_transform.bin')

In [ ]:
q_had_ref = q_had.reshape(-1, n3).to(torch.float32)

In [ ]:
for i in range(q_had_ref.shape[0]):
    for j in range(q_had_ref.shape[1]):
        hex_value = float_to_hex(q_had_ref[i, j].item())
        print(f"0x{hex_value}", end=" ")
    print()

In [ ]:
a = torch.tensor([1100.0, -1110.1, 7.6, 8.4], dtype=torch.float32)

In [ ]:
a.to(torch.int8)